# CPLEX CPのブラックボックス最適化のデモ
Module docplex.cp.blackbox<br>
https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.blackbox.py.html?highlight=blackbox#module-docplex.cp.blackbox

CP オプティマイザーの新しいブラック・ボックス最適化機能 - IBM Documentation <br>
https://www.ibm.com/docs/ja/icos/20.1.0?topic=2010-new-cp-optimizer-black-box-optimization-feature

CP Optimizer black-box expressions in Python - IBM Documentation <br>
https://www.ibm.com/docs/ja/icos/22.1.1?topic=SSSA5P_22.1.1/ilog.odms.studio.help/CP_Optimizer/Release_notes/topics/relnotes_V221_blackbox_python.htm

In [52]:
# データの読み込み
import pandas as pd
#!wget 'https://raw.githubusercontent.com/yoichiro0903n/blue/main/cars.csv'
df = pd.read_csv('carsWoNan.csv')
df

,mpg,engine,horsepower,weight,acceleration
0,18,307.0,130,3504,12.0
1,15,350.0,165,3693,11.5
2,18,318.0,150,3436,11.0
3,16,304.0,150,3433,12.0
4,17,302.0,140,3449,10.5
...,...,...,...,...,...
387,34,108.0,70,2245,16.9
388,38,91.0,67,1965,15.0
389,32,91.0,67,1965,15.7
390,38,91.0,67,1995,16.2


In [54]:
# 線形回帰
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# 説明変数：燃費、エンジン体積、重量、加速性
X = df[['mpg', 'engine', 'weight', 'acceleration']] .to_numpy().tolist()
# 目的変数：馬力
Y = df['horsepower'].to_numpy().tolist()

# 線形回帰のモデルを作成
lr = LinearRegression()
lr.fit(X, Y)
print('coefficient = ', lr.coef_)
print('intercept = ', lr.intercept_)

coefficient =  [-0.39658708  0.10224376  0.01800971 -4.72344381]
intercept =  113.6089240375573


# 線形問題としてパラメーター（説明変数）を最適化する

In [65]:
# 線形問題としてパラメーター（説明変数）を最適化する
# MPを利用
from docplex.mp.model import Model
m = Model(name='Linear')

# 決定変数＝説明変数。上限下限は元データより定義
mpg = m.continuous_var(df['mpg'].min(), df['mpg'].max(), "mpg")
engine = m.continuous_var(df['engine'].min(), df['engine'].max(), "engine")
weight = m.continuous_var(df['weight'].min(), df['weight'].max(), "weight")
acceleration = m.continuous_var(df['acceleration'].min(), df['acceleration'].max(), "acceleration")
# 目的関数：目的変数の最大化
m.maximize(mpg * -0.39658708 + engine * 0.10224376 + weight * 0.01800971 + acceleration * -4.72344381 + 113.6089240375573)
# 求解：線形問題
s = m.solve()
m.print_solution()

objective: 211.343
status: OPTIMAL_SOLUTION(2)
  mpg=9.000
  engine=455.000
  weight=5140.000
  acceleration=8.000


# 回帰モデルをブラックボックス関数としてパラメーター（説明変数）最適化する

In [66]:
# 線形回帰を説明変数を引数として目的変数を返す関数として定義
# lr.predict([[10,10,10,10]])
def predictHP(mpg, engine, weight, acceleration):
    return lr.predict([[mpg, engine, weight, acceleration]])[0]


# テスト実行
predictHP(10, 10, 10, 10)

63.611149954052415

In [69]:
# 制約問題としてパラメーター（説明変数）を最適化する
# CPを利用
from docplex.cp.model import *
import docplex.cp.solver.solver as solver
# ブラックボックス関数として線形回帰の関数を定義
bbf = CpoBlackboxFunction(predictHP)
mdl = CpoModel()
# 決定変数＝説明変数。上限下限は元データより定義。CPは浮動小数点は扱えないので整数化
mpg = integer_var(df['mpg'].min(), df['mpg'].max(), "mpg")
engine = integer_var(int(df['engine'].min()), int(df['engine'].max()), "engine")
weight = integer_var(df['weight'].min(), df['weight'].max(), "weight")
acceleration = integer_var(int(df['acceleration'].min()), int(df['acceleration'].max()), "acceleration")
# 目的関数：目的変数の最大化。ブラックボックス関数を呼び出し
obj = bbf(mpg, engine, weight, acceleration)
mdl.maximize(obj)
# 求解：CP問題
msol = mdl.solve(Workers=1, TimeLimit=5)
msol.print_solution()

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Maximization problem - 4 variables, 0 constraints, 1 blackbox
 ! TimeLimit            = 5
 ! Workers              = 1
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 29.7 (before), 29.7 (after)


 !  . Memory usage      : 275.4 kB (before), 275.4 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0          4                 -
 + New bound is 1.797693e+308
 *      108.2543      816  0.13s               (gap  > 10000%)
 *      145.1055      824  0.13s               (gap  > 10000%)
        145.1055     1000          1             23 != acceleration
 *      189.7611     1053  0.15s               (gap  > 10000%)
 *      191.9457     1290  0.16s               (gap  > 10000%)
 *      198.4070     1700  0.23s               (gap  > 10000%)
        198.4070     2000          1           1859 != weight
        198.4070     3000          1        F  4547  = weight
 *      198.6521     3329  0.49s               (gap  > 10000%)
        198.6521     4000          1        F  3390  = weight
 *      199.0487     4383  0.62s               (g

## 回帰式をブラックボックス関数として記述して実行 
同じことを回帰式として展開

In [70]:
# 制約問題としてパラメーター（説明変数）を最適化する
# CPを利用
from docplex.cp.model import *
import docplex.cp.solver.solver as solver
# ブラックボックス関数として線形回帰の式を定義
bbf = CpoBlackboxFunction(lambda mpg, engine, weight, acceleration:
                          mpg * -0.39658708 + engine * 0.10224376 + weight * 0.01800971 + acceleration * -4.72344381 + 113.6089240375573)
mdl = CpoModel()
# 決定変数＝説明変数。上限下限は元データより定義。CPは浮動小数点は扱えないので整数化
mpg = integer_var(9, 46, "mpg")
engine = integer_var(68, 455, "engine")
weight = integer_var(1613, 5140, "weight")
acceleration = integer_var(8, 25, "acceleration")
# 目的関数：目的変数の最大化。ブラックボックス関数を呼び出し
obj = bbf(mpg, engine, weight, acceleration)
mdl.maximize(obj)
# 求解：CP問題
msol = mdl.solve(Workers=1, TimeLimit=5)
msol.print_solution()

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Maximization problem - 4 variables, 0 constraints, 1 blackbox
 ! TimeLimit            = 5
 ! Workers              = 1
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 29.8 (before), 29.8 (after)
 !  . Memory usage      : 275.4 kB (before), 275.4 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0          4                 -
 + New bound is 1.797693e+308


 *      108.2543      816  0.14s               (gap  > 10000%)
 *      111.4841      820  0.14s               (gap  > 10000%)
 *      142.7559      828  0.14s               (gap  > 10000%)
        142.7559     1000          1        F  1702  = weight
 *      144.7113     1471  0.18s               (gap  > 10000%)
        144.7113     2000          1             20 != mpg
 *      149.4347     2504  0.28s               (gap  > 10000%)
 *      195.6863     2914  0.33s               (gap  > 10000%)
        195.6863     3000          1        F   248  = engine
 *      196.6691     3357  0.36s               (gap  > 10000%)
        198.1473     4000          1            419  = engine
 *      198.1473     4000  0.40s               (gap  > 10000%)
        198.1473     4000          1        F        -
 *      198.2555     4384  0.43s               (gap  > 10000%)
        198.2555     5000          1           4982 != weight
        198.2555     6000          1           2321 != weight
 *      1

# 制約問題として式化できないランダムフォレストモデルをブラックボックス関数としてパラメーター（説明変数）最適化する。

In [75]:
# ランダムフォレスト
from sklearn.ensemble import RandomForestRegressor
# 説明変数：燃費、エンジン体積、重量、加速性
X = df[['mpg', 'engine', 'weight', 'acceleration']] .to_numpy().tolist()
# 目的変数：馬力
Y = df['horsepower'].to_numpy().tolist()
# ランダムフォレストのモデルを作成
forest = RandomForestRegressor()
forest.fit(X, Y)

# ランダムフォレストのモデルを関数化
def predictHP(mpg, engine, weight, acceleration):
    return forest.predict([[mpg, engine, weight, acceleration]])[0]

# テスト実行
predictHP(10, 10, 10, 10)

75.99

In [76]:
# 制約問題としてパラメーター（説明変数）を最適化する
# CPを利用
from docplex.cp.model import *
import docplex.cp.solver.solver as solver
# ブラックボックス関数としてランダムフォレストの関数を定義
bbf = CpoBlackboxFunction(predictHP)
mdl = CpoModel()
# 決定変数＝説明変数。上限下限は元データより定義。CPは浮動小数点は扱えないので整数化
mpg = integer_var(df['mpg'].min(), df['mpg'].max(), "mpg")
engine = integer_var(int(df['engine'].min()), int(df['engine'].max()), "engine")
weight = integer_var(df['weight'].min(), df['weight'].max(), "weight")
acceleration = integer_var(int(df['acceleration'].min()), int(df['acceleration'].max()), "acceleration")

# 目的関数：目的変数の最大化。ブラックボックス関数を呼び出し
obj = bbf(mpg, engine, weight, acceleration)
mdl.maximize(obj)
# 求解：CP問題
msol = mdl.solve(Workers=1, TimeLimit=5)
msol.print_solution()

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Maximization problem - 4 variables, 0 constraints, 1 blackbox
 ! TimeLimit            = 5
 ! Workers              = 1
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 29.7 (before), 29.7 (after)
 !  . Memory usage      : 275.4 kB (before), 275.4 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0          4                 -
 + New bound is 1.797693e+308


 *      75.98999      816  2.08s               (gap  > 10000%)
 *      161.6599      820  2.08s               (gap  > 10000%)
 *      180.2299      824  2.09s               (gap  > 10000%)
 *      208.5900      836  2.10s               (gap  > 10000%)
        208.5900     1000          1        F  1693  = weight
 *      209.7800     1663  3.27s               (gap  > 10000%)
        209.7800     2000          1        F  1744  = weight
 ! ----------------------------------------------------------------------------
 ! Search terminated by limit, 5 solutions found.
 ! Best objective         : 209.7800 (gap  > 10000%)
 ! Best bound             : 1.797693e+308
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 2363
 ! Number of fails        : 1152
 ! Number of b-box calls  : 960
 ! Total memory usage     : 1.1 MB (1.0 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 5.01s (5.00s engine + 0.01s extraction)
 ! Search speed 

In [64]:
import cplex
import docplex
print(sys.version)
print('cplex:' + cplex.__version__)
print('docplex:' + docplex.__version__)

3.10.9 (tags/v3.10.9:1dd9be6, Dec  6 2022, 20:01:21) [MSC v.1934 64 bit (AMD64)]
cplex:22.1.1.0
docplex:2.25.236
